In [1]:
# An example of building up the structure of a figure as a Python dictionary
# and then constructing a graph object figure from that dictionary.

In [2]:
# https://plotly.com/python/animations/#using-a-slider-and-buttons
# https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv

In [3]:
import pandas as pd

In [4]:
dataset = pd.read_csv('output_data/time_series/total-daily-tbl.csv')

In [5]:
# https://community.plotly.com/t/plotly-doesnt-load-most-of-the-time/32095/3

import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'chrome'

In [6]:
years = list(dataset['Date_Confirmed'].unique())

In [7]:
# Make list of regions

regions = []

for region in dataset['Region']:
    if region not in regions:
        regions.append(region)

In [8]:
# Make figure

fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

In [9]:
# Fill in most of layout

fig_dict['layout']['xaxis'] = {'title': 'casos totales', 'type': 'log'}
fig_dict['layout']['yaxis'] = {'title': 'casos diarios', 'type': 'log'}
fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '15-02-2020',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
    }
fig_dict['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 300,
                                                                    'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Fecha:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [10]:
# Make data

year = '15-02-2020'

for region in regions:
    
    dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
    dataset_by_year_and_region = dataset_by_year[dataset_by_year['Region'] == region]
    
    data_dict = {
        'x': list(dataset_by_year_and_region['total']),
        'y': list(dataset_by_year_and_region['daily']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_region['Region']),
    }
    fig_dict['data'].append(data_dict)

In [11]:
# Make frames

for year in years:
    
    frame = {'data': [], 'name': str(year)}
    
    for region in regions:
        
        dataset_by_year = dataset[dataset['Date_Confirmed'] == year]
        dataset_by_year_and_region = dataset_by_year[
            dataset_by_year['Region'] == region]
        
        data_dict = {
                    'x': list(dataset_by_year_and_region['total']),
                    'y': list(dataset_by_year_and_region['daily']),
                    'mode': 'lines',
                    'text': list(dataset_by_year_and_region['Region']),
                    'name': region
            }
        frame['data'].append(data_dict)
        
    fig_dict['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
         'transition': {'duration': 300}}
    ],
        'label': year,
        'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
    
fig_dict['layout']['sliders'] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()